In [281]:
#Library
import requests
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 1000)
pd.set_option("display.precision", 12)
import warnings
warnings.filterwarnings('once')

In [282]:
def get_data(endpointApi, date=None, full_response=False, columns=list()):
    URL = 'https://dolphin.jump-technology.com:8443/api/v1/'
    AUTH = ('EPITA_GROUPE10', 'LRNSpHqMgGv86AH5')
    
    payload = {'date': date, 'fullResponse': full_response}
    res = requests.get(URL + endpointApi,# + columns_to_str(columns),
                      params=payload,
                      auth=AUTH,
                      verify=False)
    return res.content.decode('utf-8')

In [283]:
def df_normalize(data):
        df = pd.json_normalize(json.loads(data))

        cols = [col for col in df.columns if col.split('.')[-1] != 'type']
        df = df[cols]

        df.columns = df.columns.map(lambda x: x.split(".")[0])
        return df

Création de la table des taux en EUR : 

on pourrait optimiser encore le nombre de requete API avec un appel pour une currency source seulement si on en a besoin et qu'il n'est pas encore dans notre dataframe
 
[Est-ce qu'on donne la posibilité de changer la currency dst en fonction de la currency du portfolio ?]

In [284]:
def getCurrencyRate():
    d = []
    currency_json = json.loads(get_data('currency'))
    for currency in currency_json:
        currency_id = currency.get('id')
        rate = get_data('currency/rate/{}/to/EUR'.format(currency_id))
        if len(rate) == 0:
            continue
        rate_value = json.loads(rate)['rate']['value']
        d.append([currency_id, rate_value])
    return pd.DataFrame(d, columns=['currency src', 'rate'])
    
df_currency_rate = getCurrencyRate()
df_currency_rate

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,currency src,rate
0,USD,"0,851208716377"
1,EUR,1
2,JPY,"0,008066467694"
3,BEF,"0,024789352477"
4,CYP,"1,708601441376"
...,...,...
23,LUF,"0,024789352477"
24,VAL,"0,000516456899"
25,SML,"0,000516456899"
26,EUr,"0,01"


[TEST] Taux de convertion entre USD et EUR :

In [285]:
def currencyRateToEUR(cur_src, df_currency_rate):
    return df_currency_rate[df_currency_rate['currency src'] == cur_src]['rate'].values[0]
rate = currencyRateToEUR('USD', df_currency_rate)
rate


'0,851208716377'

[TEST] La database de tous les asset :

In [286]:
data = get_data('asset/')
data = json.loads(data)
df = pd.json_normalize(data)
cols = [col for col in df.columns if col.split('.')[-1] != 'type']
df = df[cols]
df.columns = df.columns.map(lambda x: x.split(".")[0])

df_ETF = df
df_ETF

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,MARKET_PLACE_MIC,CODE_CIC,IS_PUBLICLY_TRADED,LAST_GROSS_VALUE,LAST_QUOTE_DATE,MARKET_PLACE_CURRENCY,CITY_OR_ACRONYM_OR_NAME,CATEGORY_CIC,CREATION_DATE,FIRST_QUOTE_DATE,LAST_QUOTE_SOURCE,CURRENCY,VL_DELAY,SUB_CATEGORY_CIC,MARKET_CAP_CATEGORY,IS_ACCESSIBILITY_PUBLIC,INDEXED_RATE,GRAPH_PAST_WEEK,GRAPH_LAST_YEAR,PORT_LINE_COUNT,MARKET_PLACE_CITY,ISIN_CODE,IS_PORTFOLIO,DAILY_PRICE_COUNT,SUB_SECTOR,COUNTRY,quote_value_type,PAY_ACCOUNT_ON_DEFAULT,LAST_QUOTES_UPDATE_DATE,BLOOMBERG_CODE,MARKET_PLACE_DESC,GRAPH_PERIOD,INDUSTRY,LIQUIDITY_ALGO,GRAPH_PAST_YEAR,REST_OBJECT_ID,IS_STRUCTURED_PRODUCT,asset_custom_classification_bloomberg,COUNTRY_CIC,GLOBAL_TYPE,ASSET_OR_MARKET_PLACE_COUNTRY,ASSET_STATUS,LAST_CLOSE_VALUE_IN_CURR,BLOOMBERG_ID,MARKET_PLACE_TELEKURS,LAST_CLOSE_VALUE,MODIFICATION_DATE,TYPE,GICS_INDUSTRY_GROUP,PORT_CONTENT_COUNT,IS_FUND_FED,COUNTRY_ISO_3,FINAL_UNDERLYING,GICS_SUB_INDUSTRY,UNDERLYING_OR_ASSET_IF_UNDERLYING_NOT_AVAILABLE,GICS_SECTOR,FLOATING_VALUE,GRAPH_LAST_6_MONTH,asset_attr_init,MARKET_PLACE,MARKET_PLACE_OR_ASSET_COUNTRY,MARKET_PLACE_COUNTRY,LABEL,IS_EURONEXT,SUB_TYPE,IS_COMPOSITE,FINAL_UNDERLYING_TYPE,IS_FUND_SAVE_SENS,GICS_INDUSTRY,EMIR_CLEARING_ID_TYPE,SECTOR,IS_DEPOSITARY_QUOTES,REST_UNIQUE_NUMBER,SEDOL_CODE,QUOTE_TYPE,ASSET PRICE MODE,ASSET_DATABASE_ID,IS_EUROLAND,MARKET_PLACE_TYPE,SUP_SECTOR,STOCK_TYPE,asset_custom_classification_Tab,ELIGIBLE_PEA,ELIGIBLE_PEA_PME,ASSET_MARKET_SUB_GROUP,asset_fund_info_asset_class_focus,BOND_WAM_COMPUTATION_MODE,asset_fund_info_strategy,BOND_CREDIT_RISK_COMPUTATION_MODE,asset_fund_info_maximum_front_end_fee,BOND_RISK_COMPUTATION_MODE,asset_fund_info_bloomberg_ticker,asset_fund_info_is_feeded,asset_fund_info_authorized_for_retail_investors,asset_fund_info_fund_leverage,BOND_YTM_COMPUTATION_MODE,asset_fund_info_objective_long,asset_fund_info_is_euro_ucits,asset_fund_info_administrator,TRANSPARENCY_LIST,asset_fund_info_isin_code,asset_fund_info_minimum_investment_shares,asset_fund_info_depositary,asset_fund_info_manager_stated_fee,BOND_WAL_COMPUTATION_MODE,asset_fund_info_cut_off,asset_fund_info_maximum_back_end_fee,asset_fund_info_company,asset_fund_info_vl_date,IS_REAL_ESTATE_FUND,asset_fund_info_manager_stated_perf_fee,asset_fund_info_auditor,FUND_TYPE,asset_fund_info_fund_type,asset_fund_info_valorisation_frequency,asset_fund_info_creation_date,MIN_BUY_INITIAL_AMOUNT,MIN_BUY_AMOUNT,asset_fund_info_pct_max_opcvm,asset_fund_info_initial_minimum_subscription,asset_fund_info_fund_open_pymt_shr,asset_fund_info_amf_classification,asset_fund_info_decimalisation,asset_fund_info_amf_date,asset_fund_info_minimum_subscription,asset_fund_info_day_cut_off,asset_fund_info_vl_known,asset_fund_info_geographical_focus,asset_fund_info_subscription_committee,asset_fund_info_abide_with_the_four_criterias,asset_fund_info_reuters_code,REUTERS_CODE,asset_fund_info_subscription_frequency,asset_fund_info_redemption_frequency,VALOREN_CODE,asset_fund_info_valoren_code,asset_fund_info_manager,asset_fund_info_redemption_committee,asset_fund_info_advised_horizon,FUND_REF_IDX,asset_fund_info_comanager,HAS_TOBIN_TAX,CUSIP_CODE,LOCAL_CODE,asset_fund_info_rate_entry,asset_fund_info_management_style,asset_fund_info_rate_output,asset_fund_info_rate_entry_sdg,asset_fund_info_rate_output_sdg,MARKET_QUOTE_IDENTIFIER,asset_fund_info_manager_committee,PORT_ASSET_CONTENT_COUNT,INDEX_TYPE,TELEKURS_QUOTE_PLACE_CODE,QUOTE_PLACE,IS_SRD,PORT_BASE_NAV,PORT_VL,REAL_TIME_PORT_CASH,PORT_CLEAN_MODE,PORT_CASH,PORT_NET_EX_CASH_EQUIVALENT,NB_COMPTE_ON_PORTFOLIO,PORT_CURRENCY_HEDGE,LOCK_MODE,EAI_FRONTPORT_CASH_BY_CRCY,PORT_IS_HISTORICAL,PORT_CASH_AVAILABLE,PORT_TYPE,PORT_FEED_TYPE,EAI_BACKPORT_CASH_BY_CRCY,EAI_PORTFOLIO_FEED_NAME,PORT_NET_EX_CASH,PORT_NEED_RECONCILIATION,PORT_CASH_EQUIVALENTS,PORT_FEED_CODE,EAI_PORT_AVAILABLE_CASH,PORT_STATUS,PRICER_MODE,VL_DELAY_VALUE,PORT_RATING_MOODYS_SHORT_TERM,PORT_PERCENT_EQUIVALENT,PORT_RATING_SP_SHORT_TERM,PORT_FIRST_TRANSACTION,PORT_RATING_MOODYS,PORT_RATING_SP,PORT_PERCENT_CASH,PORT_RAT

Récupération de la liste d'actifs en base :

[On prend juste ces valeurs comme dans l'exemple ou toutes les valeurs des assets ?]

In [287]:
def get_df_asset(asset_date, asset_id='ASSET_DATABASE_ID', asset_label='LABEL',
                 asset_type='TYPE', asset_last_close_value='LAST_CLOSE_VALUE_IN_CURR'):
    data = get_data('asset?columns={}&columns={}&columns={}&columns={}&date={}'
                   .format(asset_id, asset_label, asset_type, asset_last_close_value, asset_date))
    df_asset = df_normalize(data)
    return df_asset

df_ETF = get_df_asset('2013-06-14')
df_ETF

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,TYPE,LAST_CLOSE_VALUE_IN_CURR,LABEL,ASSET_DATABASE_ID
0,STOCK,"12,12 EUR",1&1 DRILLISCH AG,1845
1,STOCK,"44,91 EUR",1000MERCIS,1846
2,FUND,"30,457 EUR",1818-VEGA EUR CO,2122
3,FUND,"1342,23 EUR",29 HAUSSMANN CROISSANCE EUROPE D,2123
4,FUND,NaN,29 HAUSSMANN EQUILIBRE,2124
...,...,...,...,...
475,STOCK,"8,53 EUR",YAMANA GOLD INC,1879
476,STOCK,"7,2 EUR",YMAGIS SA,1997
477,STOCK,"2,49 EUR",ZCCM INVESTMEN - B,1991
478,STOCK,"30,8 USD",ZOETIS INC,2106


Différence entre LAST_CLOSE_VALUE_IN_CURR et LAST_CLOSE_VALUE ??


Conversion des valeurs monétaires en EUR :

In [288]:
def column_convert_currency(df_ETF, col, df_currency_rate):
    new_col = []
    for elt in df_ETF[col]:
        if elt is np.nan:
            new_col.append(elt)
            continue
            
        value, currency_name = elt.split(' ')
        rate = currencyRateToEUR(currency_name, df_currency_rate)

        new_value = float(value.replace(',', '.')) * float(rate.replace(',', '.'))
        new_col.append(new_value)
    df_ETF[col] = new_col
    return df_ETF

def df_convert_currency(df_ETF, df_currency_rate):
    """cols_name_to_convert = ['LAST_GROSS_VALUE',
                            'LAST_CLOSE_VALUE_IN_CURR',
                            'LAST_CLOSE_VALUE',
                            'MIN_BUY_AMOUNT',
                            'asset_fund_info_minimum_subscription',
                            'MIN_BUY_INITIAL_AMOUNT'
                           ]"""
    cols_name_to_convert = ['LAST_CLOSE_VALUE_IN_CURR']
    for col in cols_name_to_convert:
        df_ETF = column_convert_currency(df_ETF, col, df_currency_rate)
    return df_ETF

df_ETF_convert = df_convert_currency(df_ETF, df_currency_rate)
df_ETF_convert

,TYPE,LAST_CLOSE_VALUE_IN_CURR,LABEL,ASSET_DATABASE_ID
0,STOCK,12.120000000000,1&1 DRILLISCH AG,1845
1,STOCK,44.910000000000,1000MERCIS,1846
2,FUND,30.457000000000,1818-VEGA EUR CO,2122
3,FUND,1342.230000000000,29 HAUSSMANN CROISSANCE EUROPE D,2123
4,FUND,NaN,29 HAUSSMANN EQUILIBRE,2124
...,...,...,...,...
475,STOCK,8.530000000000,YAMANA GOLD INC,1879
476,STOCK,7.200000000000,YMAGIS SA,1997
477,STOCK,2.490000000000,ZCCM INVESTMEN - B,1991
478,STOCK,26.217228464412,ZOETIS INC,2106


Récupération de la composition d’un portefeuille :

In [289]:
data = get_data('portfolio/')
print(data)

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Récupération des valeurs de cotation:

Cotation de l'actif 1845 en fonction de l'id et de la date de debut et fin :

In [290]:
def get_cotation_values(asset_id, start_date, end_date):
    data = get_data('asset/{}/quote?start_date={}&end_date={}'
                    .format(asset_id, start_date, end_date))
    df_cotation = df_normalize(data)
    
    df_cotation.columns = df_cotation.columns.map(lambda x: x.split(".")[0])
    return df_cotation

df_cotation = get_cotation_values('1845', '2013-06-14', '2019-05-31')
df_cotation
    

,date,nav,gross,volume,high,real_close_price,feed_source,low,ask,asset,pl,bid,close,return,open
0,2013-06-14,"12,12","12,12",312048,"12,17","12,12",-10,"12,01","12,13",1845,"0,11","12,075","12,12","0,009159034138","12,01"
1,2013-06-17,"12,61","12,61",447491,"12,65","12,61",-10,"12,085","12,655",1845,"0,49","12,575","12,61","0,040429042904","12,085"
2,2013-06-18,"12,88","12,88",300052,"12,88","12,88",-10,"12,555","12,905",1845,"0,27","12,825","12,88","0,021411578113","12,605"
3,2013-06-19,"12,815","12,815",260988,"12,96","12,815",-10,"12,71","12,805",1845,"-0,065","12,795","12,815","-0,005046583851","12,865"
4,2013-06-20,"12,37","12,37",263978,"12,775","12,37",-10,"12,37","12,435",1845,"-0,445","12,355","12,37","-0,034724931721","12,71"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1519,2019-05-27,"36,006862268298","36,006862268298",NaN,NaN,"36,006862268298",-10,NaN,NaN,1845,"0,539152338935",NaN,"36,006862268298","0,015201216543",NaN
1520,2019-05-28,"36,067253192136","36,067253192136",NaN,NaN,"36,067253192136",-10,NaN,NaN,1845,"0,060390923838",NaN,"36,067253192136","0,001677205956",NaN
1521,2019-05-29,"36,31169724569","36,31169724569",NaN,NaN,"36,31169724569",-10,NaN,NaN,1845,"0,244444053554",NaN,"36,31169724569","0,006777451342",NaN
1522,2019-05-30,"35,796349125318","35,796349125318",NaN,NaN,"35,796349125318",-10,NaN,NaN,1845,"-0,515348120373",NaN,"35,796349125318","-0,014192344601",NaN


Récupération de la liste des ratios :

In [291]:
def get_ratio():
    data = get_data('ratio')
    df_ratio = df_normalize(data)
    return df_ratio

df_ratio = get_ratio()
df_ratio

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,id,name,is_benchmark_needed,is_percent
0,11,Correlation,True,False
1,7,Exposition action,False,True
2,15,Exposition action,False,True
3,8,RDT,False,True
4,13,Rendement,False,True
5,9,Rendement_annualise,False,True
6,12,Sharpe,False,False
7,14,VaR historique,False,True
8,10,Volatilite,False,True


Récupération de la composition d’un portefeuille :


[ERROR] Marche pas

In [294]:
def get_portfolio(portfolio_id):
    data = get_data('portfolio/{}/dyn_amount_compo'.format(portfolio_id))
    df_portfolio = df_normalize(data)
    return df_portfolio

df_portfolio = get_portfolio(564)
df_portfolio

/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dolphin.jump-technology.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


,error_message
0,564 not found


[TODO] MAJ de la composition d’un portefeuille :

ATTENTION PUT pas GET

[TODO] Calcul de ratios :